In [1]:
# %% [markdown]
# # 📊 Table Data Processing for RAG Application
# Complete notebook to test table data processing with Excel and DOCX files

# %% [markdown]
# ## Step 1: Install Required Packages

# %%
# !pip install -q langchain langchain-community chromadb sentence-transformers
# !pip install -q pandas openpyxl python-docx pypdf
# !pip install -q ollama  # For local LLM

# %% [markdown]
# ## Step 2: Import Libraries

# %%
import pandas as pd
import numpy as np
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from typing import List, Dict, Optional
import json
from io import BytesIO
import tempfile
import os
from langchain_community.embeddings import OllamaEmbeddings


# For DOCX processing
try:
    from docx import Document as DocxDocument
    docx_available = True
except ImportError:
    docx_available = False
    print("python-docx not available. DOCX processing disabled.")

# For Ollama
try:
    import ollama
    ollama_available = True
except ImportError:
    ollama_available = False
    print("Ollama not available. Using mock responses.")

In [2]:
# %% [markdown]
# ## Step 3: Create Sample Table Data

# %%
# Create sample Excel data
def create_sample_excel_file():
    """Create a sample Excel file with multiple sheets"""
    
    # Sample employee data
    employee_data = {
        'Employee_ID': [101, 102, 103, 104, 105],
        'Name': ['John Smith', 'Sarah Johnson', 'Mike Brown', 'Emily Davis', 'David Wilson'],
        'Department': ['Engineering', 'Marketing', 'Engineering', 'Sales', 'Engineering'],
        'Salary': [75000, 65000, 82000, 58000, 90000],
        'Join_Date': ['2022-01-15', '2021-08-22', '2020-03-10', '2023-02-01', '2019-11-05']
    }
    
    # Sample department budget
    budget_data = {
        'Department': ['Engineering', 'Marketing', 'Sales', 'HR'],
        'Q1_Budget': [150000, 80000, 70000, 50000],
        'Q2_Budget': [160000, 85000, 75000, 52000],
        'Q3_Budget': [170000, 90000, 80000, 55000],
        'Q4_Budget': [180000, 95000, 85000, 58000]
    }
    
    # Create DataFrames
    df_employees = pd.DataFrame(employee_data)
    df_budget = pd.DataFrame(budget_data)
    
    # Create Excel file
    with pd.ExcelWriter('sample_data.xlsx', engine='openpyxl') as writer:
        df_employees.to_excel(writer, sheet_name='Employees', index=False)
        df_budget.to_excel(writer, sheet_name='Department_Budget', index=False)
        # Add a third sheet with some calculations
        summary_data = {
            'Metric': ['Total Employees', 'Avg Salary', 'Max Salary', 'Min Salary'],
            'Value': [len(df_employees), df_employees['Salary'].mean(), 
                     df_employees['Salary'].max(), df_employees['Salary'].min()]
        }
        pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
    
    print("✅ Sample Excel file created: 'sample_data.xlsx'")
    return df_employees, df_budget

# Create the sample file
df_emp, df_bud = create_sample_excel_file()


✅ Sample Excel file created: 'sample_data.xlsx'


In [ ]:

# %% [markdown]
# ## Step 5: Test the Table Processor

# %%
# Initialize the processor
processor = AdvancedTableProcessor()

# Process our sample Excel file
documents = processor.process_excel_tables("sample_data.xlsx")

print(f"📄 Generated {len(documents)} documents from Excel file")
print("\n" + "="*50)

# Display the different representations
for i, doc in enumerate(documents):
    print(f"\n📝 Document {i+1} - Type: {doc.metadata.get('type', 'unknown')}")
    print(f"Source: {doc.metadata.get('sheet_name', 'unknown')}")
    print(f"Content preview: {doc.page_content[:200]}...")
    print("-" * 30)


📄 Generated 9 documents from Excel file


📝 Document 1 - Type: table_text
Source: Employees
Content preview: # Table: Employees

| Employee_ID | Name | Department | Salary | Join_Date |
|---|---|---|---|---|
| 101 | John Smith | Engineering | 75000 | 2022-01-15 |
| 102 | Sarah Johnson | Marketing | 65000 | 2...
------------------------------

📝 Document 2 - Type: table_json
Source: Employees
Content preview: {
  "schema": {
    "columns": [
      "Employee_ID",
      "Name",
      "Department",
      "Salary",
      "Join_Date"
    ],
    "data_types": {
      "Employee_ID": "int64",
      "Name": "object...
------------------------------

📝 Document 3 - Type: schema_info
Source: Employees
Content preview: Table Schema: Employees

Dimensions: 5 rows × 5 columns

Column Analysis:
- Employee_ID (int64): min=101, max=105, mean=103.00
- Name (object): 5 unique values: John Smith, Sarah Johnson, Mike Brown, ...
------------------------------

📝 Document 4 - Type: table_text
Source: Departme

In [ ]:

# %% [markdown]
# ## Step 6: Set Up Vector Database with Embeddings

# %%
# Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-embed-text"  # Good balance of speed and quality
)

# Create vector store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory="./table_rag_db"
)

print("✅ Vector database created successfully!")
print(f"📊 Total documents in DB: {vectorstore._collection.count()}")

# %% [markdown]
# ## Step 7: Enhanced Retrieval for Tabular Data

# %%
class TableAwareRetriever:
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore
        self.table_keywords = [
            "table", "row", "column", "cell", "data", "record",
            "excel", "sheet", "spreadsheet", "tabular", "grid",
            "salary", "department", "budget", "employee", "calculate"
        ]
    
    def enhance_table_query(self, query: str) -> List[str]:
        """Expand queries to better match table content"""
        enhanced_queries = [query]
        
        # Add table-specific context
        enhanced_queries.append(f"table data: {query}")
        enhanced_queries.append(f"excel spreadsheet: {query}")
        
        # If query seems table-related, add more variations
        if any(keyword in query.lower() for keyword in self.table_keywords):
            enhanced_queries.append(f"{query} in tabular format")
            enhanced_queries.append(f"spreadsheet information for {query}")
        
        return enhanced_queries
    
    def retrieve_table_context(self, query: str, top_k: int = 5) -> str:
        """Specialized retrieval for tabular data"""
        enhanced_queries = self.enhance_table_query(query)
        
        all_results = []
        for eq in enhanced_queries:
            try:
                results = self.vectorstore.similarity_search(eq, k=top_k)
                all_results.extend(results)
            except Exception as e:
                print(f"Error in retrieval for '{eq}': {e}")
        
        if not all_results:
            return "No relevant context found."
        
        # Prioritize table content
        table_results = []
        other_results = []
        
        for result in all_results:
            doc_type = result.metadata.get("type", "")
            if "table" in doc_type or "schema" in doc_type:
                table_results.append(result)
            else:
                other_results.append(result)
        
        # Combine with tables first
        prioritized_results = table_results + other_results
        
        # Remove duplicates and limit results
        seen_content = set()
        final_results = []
        
        for result in prioritized_results:
            if result.page_content not in seen_content:
                seen_content.add(result.page_content)
                final_results.append(result)
                if len(final_results) >= top_k:
                    break
        
        # Format the context
        context_parts = []
        for i, result in enumerate(final_results):
            source = result.metadata.get("sheet_name", result.metadata.get("source", "Unknown"))
            doc_type = result.metadata.get("type", "unknown")
            context_parts.append(f"--- SOURCE {i+1}: {source} ({doc_type}) ---")
            context_parts.append(result.page_content)
            context_parts.append("")  # Empty line for separation
        
        return "\n".join(context_parts)

# Initialize retriever
retriever = TableAwareRetriever(vectorstore)

# %% [markdown]
# ## Step 8: Test Retrieval with Sample Questions

# %%
# Test questions
test_questions = [
    "What is John Smith's salary?",
    "Which department has the highest budget in Q3?",
    "How many employees are in the Engineering department?",
    "What is the average salary?",
    "Who has the highest salary?",
    "Show me the budget data for Marketing",
    "How many people were hired in 2022?",
    "What's the total Q4 budget across all departments?"
]

print("🧪 Testing Table Retrieval\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. QUESTION: {question}")
    context = retriever.retrieve_table_context(question)
    print("RETRIEVED CONTEXT:")
    print(context[:500] + "..." if len(context) > 500 else context)
    print("-" * 80)

# %% [markdown]
# ## Step 9: Enhanced Prompting for Table Questions

# %%
def create_table_aware_prompt(query: str, context: str) -> str:
    """Create specialized prompts for tabular data questions"""
    
    return f"""You are an expert data analyst. Use the following tabular data to answer the question accurately.

TABULAR DATA CONTEXT:
{context}

QUESTION: {query}

ANALYSIS INSTRUCTIONS:
1. FIRST check if the question is about the tabular data provided
2. For salary questions: Look for "Salary" column and specific names
3. For department questions: Look for "Department" column and counts/aggregates
4. For budget questions: Look for budget columns and department names
5. For date questions: Look for date columns and filter accordingly
6. If asked for calculations: Perform accurate math (sum, average, count, etc.)
7. If the data isn't available: Say "I cannot find this information in the provided data"
8. Be precise and reference specific tables when possible

FORMATTING:
- Reference the source: "According to the [sheet_name] table..."
- Show calculations: "The average is calculated as (sum of salaries / count) = result"
- Be specific: Instead of "someone", use the actual name from the data

ANSWER:"""

# Mock Ollama response for testing
def mock_ollama_response(prompt: str) -> str:
    """Mock function for testing without Ollama"""
    if "John Smith" in prompt and "salary" in prompt.lower():
        return "John Smith's salary is $75,000 according to the Employees table."
    elif "highest budget" in prompt.lower() and "q3" in prompt.lower():
        return "The Engineering department has the highest Q3 budget of $170,000 according to the Department_Budget table."
    elif "engineering" in prompt.lower() and "how many" in prompt.lower():
        return "There are 3 employees in the Engineering department: John Smith, Mike Brown, and David Wilson."
    elif "average salary" in prompt.lower():
        return "The average salary is $74,000, calculated as (75000 + 65000 + 82000 + 58000 + 90000) / 5 = 74000."
    else:
        return "I need to analyze the table data to answer this question accurately."

# %% [markdown]
# ## Step 10: Complete RAG Pipeline Test

# %%
def test_complete_rag_pipeline(questions):
    """Test the complete RAG pipeline"""
    
    print("🚀 Testing Complete RAG Pipeline")
    print("=" * 80)
    
    for i, question in enumerate(questions, 1):
        print(f"\n{i}. QUESTION: {question}")
        
        # Retrieve context
        context = retriever.retrieve_table_context(question)
        
        # Create prompt
        prompt = create_table_aware_prompt(question, context)
        
        # Generate response (using mock or real Ollama)
        if ollama_available:
            try:
                response = ollama.chat(
                    model='mistral:7b-instruct',
                    messages=[{'role': 'user', 'content': prompt}],
                    options={'temperature': 0.1}
                )
                answer = response['message']['content']
            except Exception as e:
                answer = f"Ollama error: {e}. Using mock response."
                answer += "\n" + mock_ollama_response(prompt)
        else:
            answer = mock_ollama_response(prompt)
        
        print("ANSWER:")
        print(answer)
        print("-" * 80)

# Run the complete test
test_complete_rag_pipeline(test_questions[:4])  # Test first 4 questions

# %% [markdown]
# ## Step 11: Performance Evaluation

# %%
def evaluate_rag_accuracy(questions, expected_answers):
    """Evaluate the RAG system's accuracy"""
    
    print("📊 Evaluating RAG Accuracy")
    print("=" * 80)
    
    correct_count = 0
    results = []
    
    for i, (question, expected) in enumerate(zip(questions, expected_answers)):
        print(f"\n{i+1}. QUESTION: {question}")
        
        context = retriever.retrieve_table_context(question)
        prompt = create_table_aware_prompt(question, context)
        
        if ollama_available:
            try:
                response = ollama.chat(
                    model='mistral:7b-instruct',
                    messages=[{'role': 'user', 'content': prompt}],
                    options={'temperature': 0.1}
                )
                answer = response['message']['content']
            except:
                answer = mock_ollama_response(prompt)
        else:
            answer = mock_ollama_response(prompt)
        
        # Simple accuracy check (could be enhanced)
        is_correct = any(keyword in answer.lower() for keyword in expected.lower().split()[:3])
        
        if is_correct:
            correct_count += 1
            status = "✅ CORRECT"
        else:
            status = "❌ INCORRECT"
        
        results.append({
            'question': question,
            'answer': answer,
            'expected': expected,
            'correct': is_correct
        })
        
        print(f"ANSWER: {answer}")
        print(f"EXPECTED: {expected}")
        print(f"STATUS: {status}")
        print("-" * 80)
    
    accuracy = correct_count / len(questions)
    print(f"\n🎯 FINAL ACCURACY: {accuracy:.2%} ({correct_count}/{len(questions)})")
    
    return results, accuracy

# Expected answers for our test questions
expected_answers = [
    "John Smith's salary is $75,000",
    "Engineering has the highest Q3 budget of $170,000",
    "3 employees in Engineering department",
    "The average salary is $74,000"
]

# Run evaluation
evaluation_results, accuracy = evaluate_rag_accuracy(
    test_questions[:4], 
    expected_answers
)

# %% [markdown]
# ## Step 12: Create a Simple Interactive Demo

# %%
def interactive_demo():
    """Interactive demo for table RAG"""
    print("💬 Interactive Table RAG Demo")
    print("Type 'quit' to exit\n")
    
    while True:
        question = input("\nAsk a question about the data: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
        
        if not question:
            continue
        
        # Process the question
        context = retriever.retrieve_table_context(question)
        prompt = create_table_aware_prompt(question, context)
        
        print("\n🔍 Retrieving information...")
        
        if ollama_available:
            try:
                response = ollama.chat(
                    model='mistral:7b-instruct',
                    messages=[{'role': 'user', 'content': prompt}],
                    options={'temperature': 0.1}
                )
                answer = response['message']['content']
            except Exception as e:
                answer = f"Error: {e}. Please ensure Ollama is running."
        else:
            answer = mock_ollama_response(prompt)
        
        print(f"\n🤖 ANSWER: {answer}")

# Uncomment to run interactive demo
# interactive_demo()

# %% [markdown]
# ## Step 13: Cleanup and Summary

# %%
# Cleanup function
def cleanup():
    """Clean up generated files"""
    files_to_remove = ['sample_data.xlsx']
    dirs_to_remove = ['./table_rag_db']
    
    for file in files_to_remove:
        if os.path.exists(file):
            os.remove(file)
            print(f"✅ Removed {file}")
    
    for directory in dirs_to_remove:
        if os.path.exists(directory):
            import shutil
            shutil.rmtree(directory)
            print(f"✅ Removed {directory}")

# Display summary
print("📋 RAG SYSTEM SUMMARY")
print("=" * 50)
print(f"Embedding Model: all-MiniLM-L6-v2")
print(f"Vector Database: Chroma ({vectorstore._collection.count()} documents)")
print(f"Table Processor: AdvancedTableProcessor")
print(f"LLM Available: {ollama_available}")
print(f"DOCX Support: {docx_available}")
print(f"Test Accuracy: {accuracy:.2%}")

print("\n✅ Table RAG system is ready! Key features:")
print("  - Multi-format table support (Excel, CSV, DOCX)")
print("  - Multiple table representations (text, JSON, schema)")
print("  - Enhanced table-aware retrieval")
print("  - Specialized prompting for tabular data")
print("  - Accuracy evaluation framework")

# Uncomment to cleanup
# cleanup()

In [7]:
for doc in documents:
    for k, v in doc.metadata.items():
        if isinstance(v, list):
            doc.metadata[k] = str(v)

In [8]:
# %%
# Initialize embeddings using Ollama's nomic-embed-text model
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Create vector store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory="./table_rag_db"
)

print("✅ Vector database created successfully!")
print(f"📊 Total documents in DB: {vectorstore._collection.count()}")

✅ Vector database created successfully!
📊 Total documents in DB: 9


In [10]:

# %% [markdown]
# ## Step 7: Enhanced Retrieval for Tabular Data

# %%
class TableAwareRetriever:
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore
        self.table_keywords = [
            "table", "row", "column", "cell", "data", "record",
            "excel", "sheet", "spreadsheet", "tabular", "grid",
            "salary", "department", "budget", "employee", "calculate"
        ]
    
    def enhance_table_query(self, query: str) -> List[str]:
        """Expand queries to better match table content"""
        enhanced_queries = [query]
        
        # Add table-specific context
        enhanced_queries.append(f"table data: {query}")
        enhanced_queries.append(f"excel spreadsheet: {query}")
        
        # If query seems table-related, add more variations
        if any(keyword in query.lower() for keyword in self.table_keywords):
            enhanced_queries.append(f"{query} in tabular format")
            enhanced_queries.append(f"spreadsheet information for {query}")
        
        return enhanced_queries
    
    def retrieve_table_context(self, query: str, top_k: int = 5) -> str:
        """Specialized retrieval for tabular data"""
        enhanced_queries = self.enhance_table_query(query)
        
        all_results = []
        for eq in enhanced_queries:
            try:
                results = self.vectorstore.similarity_search(eq, k=top_k)
                all_results.extend(results)
            except Exception as e:
                print(f"Error in retrieval for '{eq}': {e}")
        
        if not all_results:
            return "No relevant context found."
        
        # Prioritize table content
        table_results = []
        other_results = []
        
        for result in all_results:
            doc_type = result.metadata.get("type", "")
            if "table" in doc_type or "schema" in doc_type:
                table_results.append(result)
            else:
                other_results.append(result)
        
        # Combine with tables first
        prioritized_results = table_results + other_results
        
        # Remove duplicates and limit results
        seen_content = set()
        final_results = []
        
        for result in prioritized_results:
            if result.page_content not in seen_content:
                seen_content.add(result.page_content)
                final_results.append(result)
                if len(final_results) >= top_k:
                    break
        
        # Format the context
        context_parts = []
        for i, result in enumerate(final_results):
            source = result.metadata.get("sheet_name", result.metadata.get("source", "Unknown"))
            doc_type = result.metadata.get("type", "unknown")
            context_parts.append(f"--- SOURCE {i+1}: {source} ({doc_type}) ---")
            context_parts.append(result.page_content)
            context_parts.append("")  # Empty line for separation
        
        return "\n".join(context_parts)

# Initialize retriever
retriever = TableAwareRetriever(vectorstore)


In [11]:
# %% [markdown]
# ## Step 8: Test Retrieval with Sample Questions

# %%
# Test questions
test_questions = [
    "What is John Smith's salary?",
    "Which department has the highest budget in Q3?",
    "How many employees are in the Engineering department?",
    "What is the average salary?",
    "Who has the highest salary?",
    "Show me the budget data for Marketing",
    "How many people were hired in 2022?",
    "What's the total Q4 budget across all departments?"
]

print("🧪 Testing Table Retrieval\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. QUESTION: {question}")
    context = retriever.retrieve_table_context(question)
    print("RETRIEVED CONTEXT:")
    print(context[:500] + "..." if len(context) > 500 else context)
    print("-" * 80)


🧪 Testing Table Retrieval


1. QUESTION: What is John Smith's salary?
RETRIEVED CONTEXT:
--- SOURCE 1: Employees (table_text) ---
# Table: Employees

| Employee_ID | Name | Department | Salary | Join_Date |
|---|---|---|---|---|
| 101 | John Smith | Engineering | 75000 | 2022-01-15 |
| 102 | Sarah Johnson | Marketing | 65000 | 2021-08-22 |
| 103 | Mike Brown | Engineering | 82000 | 2020-03-10 |
| 104 | Emily Davis | Sales | 58000 | 2023-02-01 |
| 105 | David Wilson | Engineering | 90000 | 2019-11-05 |


--- SOURCE 2: Summary (table_text) ---
# Table: Summary

| Metric | Value |
|---...
--------------------------------------------------------------------------------

2. QUESTION: Which department has the highest budget in Q3?
RETRIEVED CONTEXT:
--- SOURCE 1: Department_Budget (table_text) ---
# Table: Department_Budget

| Department | Q1_Budget | Q2_Budget | Q3_Budget | Q4_Budget |
|---|---|---|---|---|
| Engineering | 150000 | 160000 | 170000 | 180000 |
| Marketing | 80000 | 85000 | 90

In [14]:
# %% [markdown]
# ## Step 9: Enhanced Prompting for Table Questions

# %%
def create_table_aware_prompt(query: str, context: str) -> str:
    """Create specialized prompts for tabular data questions"""
    
    return f"""You are an expert data analyst. Use the following tabular data to answer the question accurately.

TABULAR DATA CONTEXT:
{context}

QUESTION: {query}

ANALYSIS INSTRUCTIONS:
1. FIRST check if the question is about the tabular data provided
2. For salary questions: Look for "Salary" column and specific names
3. For department questions: Look for "Department" column and counts/aggregates
4. For budget questions: Look for budget columns and department names
5. For date questions: Look for date columns and filter accordingly
6. If asked for calculations: Perform accurate math (sum, average, count, etc.)
7. If the data isn't available: Say "I cannot find this information in the provided data"
8. Be precise and reference specific tables when possible

FORMATTING:
- Reference the source: "According to the [sheet_name] table..."
- Show calculations: "The average is calculated as (sum of salaries / count) = result"
- Be specific: Instead of "someone", use the actual name from the data

ANSWER:"""

# Mock Ollama response for testing
def mock_ollama_response(prompt: str) -> str:
    """Mock function for testing without Ollama"""
    if "John Smith" in prompt and "salary" in prompt.lower():
        return "John Smith's salary is $75,000 according to the Employees table."
    elif "highest budget" in prompt.lower() and "q3" in prompt.lower():
        return "The Engineering department has the highest Q3 budget of $170,000 according to the Department_Budget table."
    elif "engineering" in prompt.lower() and "how many" in prompt.lower():
        return "There are 3 employees in the Engineering department: John Smith, Mike Brown, and David Wilson."
    elif "average salary" in prompt.lower():
        return "The average salary is $74,000, calculated as (75000 + 65000 + 82000 + 58000 + 90000) / 5 = 74000."
    else:
        return "I need to analyze the table data to answer this question accurately."


In [ ]:

# %% [markdown]
# ## Step 10: Complete RAG Pipeline Test

# %%
def test_complete_rag_pipeline(questions):
    """Test the complete RAG pipeline"""
    
    print("🚀 Testing Complete RAG Pipeline")
    print("=" * 80)
    
    for i, question in enumerate(questions, 1):
        print(f"\n{i}. QUESTION: {question}")
        
        # Retrieve context
        context = retriever.retrieve_table_context(question)
        
        # Create prompt
        prompt = create_table_aware_prompt(question, context)
        
        # Generate response (using mock or real Ollama)
        if ollama_available:
            try:
                response = ollama.chat(
                    model='mistral',
                    messages=[{'role': 'user', 'content': prompt}],
                    options={'temperature': 0.1}
                )
                answer = response['message']['content']
            except Exception as e:
                answer = f"Ollama error: {e}. Using mock response."
                answer += "\n" + mock_ollama_response(prompt)
        else:
            answer = mock_ollama_response(prompt)
        
        print("ANSWER:")
        print(answer)
        print("-" * 80)

# Run the complete test
test_complete_rag_pipeline(test_questions[:4])  # Test first 4 questions


🚀 Testing Complete RAG Pipeline

1. QUESTION: What is John Smith's salary?


In [ ]:

# %% [markdown]
# ## Step 11: Performance Evaluation

# %%
def evaluate_rag_accuracy(questions, expected_answers):
    """Evaluate the RAG system's accuracy"""
    
    print("📊 Evaluating RAG Accuracy")
    print("=" * 80)
    
    correct_count = 0
    results = []
    
    for i, (question, expected) in enumerate(zip(questions, expected_answers)):
        print(f"\n{i+1}. QUESTION: {question}")
        
        context = retriever.retrieve_table_context(question)
        prompt = create_table_aware_prompt(question, context)
        
        if ollama_available:
            try:
                response = ollama.chat(
                    model='mistral:7b-instruct',
                    messages=[{'role': 'user', 'content': prompt}],
                    options={'temperature': 0.1}
                )
                answer = response['message']['content']
            except:
                answer = mock_ollama_response(prompt)
        else:
            answer = mock_ollama_response(prompt)
        
        # Simple accuracy check (could be enhanced)
        is_correct = any(keyword in answer.lower() for keyword in expected.lower().split()[:3])
        
        if is_correct:
            correct_count += 1
            status = "✅ CORRECT"
        else:
            status = "❌ INCORRECT"
        
        results.append({
            'question': question,
            'answer': answer,
            'expected': expected,
            'correct': is_correct
        })
        
        print(f"ANSWER: {answer}")
        print(f"EXPECTED: {expected}")
        print(f"STATUS: {status}")
        print("-" * 80)
    
    accuracy = correct_count / len(questions)
    print(f"\n🎯 FINAL ACCURACY: {accuracy:.2%} ({correct_count}/{len(questions)})")
    
    return results, accuracy

# Expected answers for our test questions
expected_answers = [
    "John Smith's salary is $75,000",
    "Engineering has the highest Q3 budget of $170,000",
    "3 employees in Engineering department",
    "The average salary is $74,000"
]

# Run evaluation
evaluation_results, accuracy = evaluate_rag_accuracy(
    test_questions[:4], 
    expected_answers
)


In [ ]:

# %% [markdown]
# ## Step 12: Create a Simple Interactive Demo

# %%
def interactive_demo():
    """Interactive demo for table RAG"""
    print("💬 Interactive Table RAG Demo")
    print("Type 'quit' to exit\n")
    
    while True:
        question = input("\nAsk a question about the data: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
        
        if not question:
            continue
        
        # Process the question
        context = retriever.retrieve_table_context(question)
        prompt = create_table_aware_prompt(question, context)
        
        print("\n🔍 Retrieving information...")
        
        if ollama_available:
            try:
                response = ollama.chat(
                    model='mistral:7b-instruct',
                    messages=[{'role': 'user', 'content': prompt}],
                    options={'temperature': 0.1}
                )
                answer = response['message']['content']
            except Exception as e:
                answer = f"Error: {e}. Please ensure Ollama is running."
        else:
            answer = mock_ollama_response(prompt)
        
        print(f"\n🤖 ANSWER: {answer}")

# Uncomment to run interactive demo
# interactive_demo()


In [ ]:

# %% [markdown]
# ## Step 13: Cleanup and Summary

# %%
# Cleanup function
def cleanup():
    """Clean up generated files"""
    files_to_remove = ['sample_data.xlsx']
    dirs_to_remove = ['./table_rag_db']
    
    for file in files_to_remove:
        if os.path.exists(file):
            os.remove(file)
            print(f"✅ Removed {file}")
    
    for directory in dirs_to_remove:
        if os.path.exists(directory):
            import shutil
            shutil.rmtree(directory)
            print(f"✅ Removed {directory}")

# Display summary
print("📋 RAG SYSTEM SUMMARY")
print("=" * 50)
print(f"Embedding Model: all-MiniLM-L6-v2")
print(f"Vector Database: Chroma ({vectorstore._collection.count()} documents)")
print(f"Table Processor: AdvancedTableProcessor")
print(f"LLM Available: {ollama_available}")
print(f"DOCX Support: {docx_available}")
print(f"Test Accuracy: {accuracy:.2%}")

print("\n✅ Table RAG system is ready! Key features:")
print("  - Multi-format table support (Excel, CSV, DOCX)")
print("  - Multiple table representations (text, JSON, schema)")
print("  - Enhanced table-aware retrieval")
print("  - Specialized prompting for tabular data")
print("  - Accuracy evaluation framework")

# Uncomment to cleanup
# cleanup()